In [1]:
import xlrd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
import json
from pandas.io.json import json_normalize
from os import path, makedirs

## Cleaning the film.xlsx file

In [2]:
ExcelFile = pd.ExcelFile('film.xlsx')

film = ExcelFile.parse('film') # Download the file

sjanger = film['Sjanger'].str.get_dummies() # Extract the sjanger column and makes it to a matrix

film = pd.concat([film.drop(['Sjanger'], axis=1), sjanger], axis=1).drop(['Unnamed: 0', 'Ukjennt'], axis=1) # Merges the film and sjanger matrix

print(film.isnull().sum())
print(film.head())

FilmID         0
Tittel         0
Action         0
Adventure      0
Animation      0
Children       0
Children's     0
Comedy         0
Crime          0
Documentary    0
Drama          0
Fantasy        0
Film-Noir      0
Horror         0
Musical        0
Mystery        0
Romance        0
Sci-Fi         0
Thriller       0
War            0
Western        0
dtype: int64
   FilmID                                    Tittel  Action  Adventure  \
0       0                 Autumn in New York (2000)       0          0   
1       1  Vie est belle, La (Life is Rosey) (1987)       0          0   
2       2                    Defying Gravity (1997)       0          0   
3       3                    Ruthless People (1986)       0          0   
4       4                  Portraits Chinois (1996)       0          0   

   Animation  Children  Children's  Comedy  Crime  Documentary  ...  Fantasy  \
0          0         0           0       0      0            0  ...        0   
1          0         0   

## Cleaning the bruker.json file

In [3]:
with open('bruker.json') as json_data:
    data = json.load(json_data) # Download the json file

data1 = pd.DataFrame.from_dict(data, orient='index')

bruker = pd.DataFrame(data.pop('data'), columns = list(data.items())[0][1]) # Makes the data into a DataFrame

print(bruker.isnull().sum()) # Checking for null values

#bruker = bruker.dropna()

#print(bruker.isnull().any().any())

bruker['Kjonn'].fillna('U', inplace = True) # Fill every null values with U for unknown
bruker['Alder'].fillna(0, inplace = True) # Fill every null values with a new value 0
bruker['Jobb'].fillna(0, inplace = True) # Fill every null values with a 0
bruker['Postkode'].fillna(0, inplace = True) # Fill every null values with a new value 0

bruker = bruker.astype({'Alder':'int64', 'Jobb':'int64'}) # Change the values to int values

print(bruker.head())

BrukerID      0
Kjonn       303
Alder       994
Jobb        593
Postkode    451
dtype: int64
   BrukerID Kjonn  Alder  Jobb    Postkode
0         0     U     45     6       92103
1         1     M     50    16  55405-2546
2         2     M     18    20       44089
3         3     M      0     1       33304
4         4     M     35     6       48105


In this file we have som null values in kjonn, alder, jobb and postkode. Since none of these columns are used in our models and predictions this will not be a problem. I have filled the null values with some other values, see the code above for details.

## Cleaning the rangering.dat file

In [4]:
rangering_dat = pd.read_csv('rangering.dat', sep='::', names=['BrukerID', 'FilmID', 'Rangering', 'Tidstempel']) # Loading the rangering file to a csv file

print(rangering_dat.isnull().sum())
rangering = rangering_dat.dropna() # Remove every column with null values
print(rangering.isnull().sum())

rangering_etter = rangering[rangering['Tidstempel'] > 965088000] # Stores all the rows with ratings after 2000
rangering_før = rangering[rangering['Tidstempel'] < 965088000] # Stores all the rows with ratings before 2000 
rangering_før['Rangering'] = rangering_før['Rangering'] /2 # Divides all the ratings before 2000 by 2 such that we get them in the form 1-5.

rangering = pd.concat([rangering_før, rangering_etter]) # Takes the ratings befor 2000 and combine it with the ratings from after 2000, and drops the column tidsstempel


<ipython-input-4-5253f71aa4f1>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  rangering_dat = pd.read_csv('rangering.dat', sep='::', names=['BrukerID', 'FilmID', 'Rangering', 'Tidstempel']) # Loading the rangering file to a csv file


BrukerID         0
FilmID           0
Rangering        0
Tidstempel    1492
dtype: int64
BrukerID      0
FilmID        0
Rangering     0
Tidstempel    0
dtype: int64


<ipython-input-4-5253f71aa4f1>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rangering_før['Rangering'] = rangering_før['Rangering'] /2 # Divides all the ratings before 2000 by 2 such that we get them in the form 1-5.


We can see that the column Tidsstempel has 1492 null values. This is a problem since we have to know if the ratings was before or after 2000. Since there are no way of figuring out these values I have decided to remove all these rows. We could have kept the rows which ratings was above 5, but I decided against this because we would then only have taken the good ratings and not the bad ones.

## Saving the new DataFrames as csv

In [5]:
if path.exists('cleandata'):
    print()
else:
    makedirs('cleandata')


film.to_csv('cleandata/film.csv', index=False)
bruker.to_csv('cleandata/bruker.csv', index=False)
rangering.to_csv('cleandata/rangering.csv', index=False)